<center> <h3>Content-based Filtering</h3></center>

This technology recommends by comparing similarities or relevance between products. This method ignores the user's purchase behavior and only considers the similarity between the products.

In [6]:
import pandas as pd
## !pip install neo4j

In [7]:
from neo4j import GraphDatabase
username = str(input("Please input your neo4j username (neo4j is default): "))
password = str(input("Please input your password: "))

Please input your neo4j username (neo4j is default): neo4j
Please input your password: Aa1234567


In [8]:
# Establish the connection to the graph db.
graphdb = GraphDatabase.driver(uri="bolt://localhost:7687", auth=(username, password))
session = graphdb.session()

<h4>User Input</h4>

In [11]:
name = str(input('What is your name?'))
favouriteMove = str(input('What is your favourite move? For example, Strange Days.'))

# Input the number of movies you want to rate.
n = int(input("How many movies you want to rate?"))

# Input movvies' titles and ratings
movie_list = dict()
for i in range(n):
    data = input("""Please enter movie title and rating score, inclue your favouriate movie.
    For example, Frozen:8, Insomnia:4, Rubber:3, Inception:5, Kite:6.""")
    temp = data.split(':')
    movie_list[temp[0]] = int(temp[1])

What is your name?Xiang Zhu
What is your favourite move?, for example, Strange DaysStrange Days
How many movies you want to rate?4
Please enter movie title and rating score, inclue your favouriate movie.
    For example, Frozen:8, Insomnia:4, Rubber:3, Inception:5, Kite:6Strange Days:10
Please enter movie title and rating score, inclue your favouriate movie.
    For example, Frozen:8, Insomnia:4, Rubber:3, Inception:5, Kite:6Frozen:8
Please enter movie title and rating score, inclue your favouriate movie.
    For example, Frozen:8, Insomnia:4, Rubber:3, Inception:5, Kite:6Insomnia:4
Please enter movie title and rating score, inclue your favouriate movie.
    For example, Frozen:8, Insomnia:4, Rubber:3, Inception:5, Kite:6Rubber:3


In [12]:
# Create user node to the database
session.run('CREATE (p:User {{name:"{}"}})'.format(name))

# Create a rating relationship between users and movies
q0 = """MATCH (p:User),(m:Movie)
WHERE p.name = '{0}' AND m.title = '{1}'
CREATE (p)-[r:RATED {{rating:{2}}}]->(m)
RETURN type(r)"""

# Establish a rating relationship
for key, value in movie_list.items():
	session.run(q0.format(name, key, value))

<h4>1. Similarity Based on Common Genres</h4>

First find the m movie, and then find out the genre (g: Genre) through the relationship (: IN_GENRE). The direction of the relationship is->. After finding (g: GENRE), reverse query <-all movies with this genre.

In [13]:
q1 = """MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)<-[:IN_GENRE]-(rec:Movie)
WHERE m.title = "{}"
WITH rec, COLLECT(g.name) AS genres, COUNT(*) AS commonGenres
RETURN rec.title, genres, commonGenres
ORDER BY commonGenres DESC LIMIT 10""".format(favouriteMove)

recommentation1 = session.run(q1)

In [14]:
movieTitle1 = []
movieGenres = []
commenGenres = []

for node in recommentation1:
    movieTitle1.append(node[0])
    movieGenres.append(node[1])
    commenGenres.append(node[2])
    
df1 = pd.DataFrame(columns = ["Movie Title", "Movie Genres", "CommenGenres"])

for i in range(0, len(movieTitle1)):
    df1 = df1.append({"Movie Title":movieTitle1[i], "Movie Genres":movieGenres[i], 
                      "CommenGenres":commenGenres[i]}, ignore_index=True)
df1

,Movie Title,Movie Genres,CommenGenres
0,Inception,"[Drama, Action, Crime, Thriller, Mystery, Sci-Fi]",6
1,Patlabor: The Movie (Kidô keisatsu patorebâ: T...,"[Drama, Action, Crime, Thriller, Mystery, Sci-Fi]",6
2,"Double, The","[Drama, Action, Crime, Thriller, Mystery]",5
3,"Girl Who Played with Fire, The (Flickan som le...","[Drama, Action, Crime, Thriller, Mystery]",5
4,Sherlock: The Abominable Bride,"[Drama, Action, Crime, Thriller, Mystery]",5
5,Kite,"[Drama, Action, Crime, Thriller, Mystery]",5
6,Source Code,"[Drama, Action, Thriller, Mystery, Sci-Fi]",5
7,Blackhat,"[Drama, Action, Crime, Thriller, Mystery]",5
8,Rubber,"[Drama, Action, Crime, Thriller, Mystery]",5
9,Knowing,"[Drama, Action, Thriller, Mystery, Sci-Fi]",5


<h4>2. Personalized Recommendations Based on Genres</h4>

If we know which movies the user has watched, we can use this information to recommend similar movies.

1. Determine the user

2. Find out the movies m rated by users

3. Find out the genre of the rated movie (g: Genre)

4. Go back through the genre g to search for movies belonging to the genre (rec: Movie), and the part of the movie that Angelica has not rated

5. Return to the movie and score based on the number of genre names —— [g.name, COUNT (*)] AS scores

6. List the data in the Scores list ——COLLECT (scores) AS scoreComponents,

7. From the Scores list, extract and add the number of names as a score-REDUCE (s = 0, x in COLLECT (scores) | s + x [1]) AS score

8. Sort by score

In [15]:
q2 = """MATCH (u:User {{name:"{}"}})-[r:RATED]->(m:Movie),
  (m)-[:IN_GENRE]->(g:Genre)<-[:IN_GENRE]-(rec:Movie)
WHERE NOT EXISTS( (u)-[:RATED]->(rec) )
WITH rec, [g.name, COUNT(*)] AS scores
RETURN rec.title AS movieTitle, rec.year AS year,
COLLECT(scores) AS scoreComponents,
REDUCE (s=0,x in COLLECT(scores) | s+x[1]) AS score
ORDER BY score DESC LIMIT 10""".format(name)

recommentation2 = session.run(q2)

In [16]:
movieTitle2 = []
year = []
scoreComponents = []
score = []

for node in recommentation2:
    movieTitle2.append(node[0])
    year.append(node[1])
    scoreComponents.append(node[2])
    score.append(node[3])
    
df2 = pd.DataFrame(columns = ["Movie Title", "Year", "Score Components", "Score"])

for i in range(0, len(movieTitle2)):
    df2 = df2.append({"Movie Title":movieTitle2[i], "Year":year[i], 
                      "Score Components":scoreComponents[i],
                      "Score":score[i]}, ignore_index=True)
df2

,Movie Title,Year,Score Components,Score
0,Motorama,1991,"[[Adventure, 18], [Comedy, 18], [Fantasy, 9], ...",161
1,Patlabor: The Movie (Kidô keisatsu patorebâ: T...,1989,"[[Animation, 9], [Drama, 28], [Action, 23], [C...",157
2,"Seven-Per-Cent Solution, The",1976,"[[Adventure, 18], [Comedy, 18], [Drama, 28], [...",143
3,Osmosis Jones,2001,"[[Animation, 9], [Comedy, 18], [Romance, 9], [...",138
4,Interstate 60,2002,"[[Adventure, 18], [Comedy, 18], [Fantasy, 9], ...",138
5,Sherlock Holmes: A Game of Shadows,2011,"[[Adventure, 18], [Comedy, 18], [Action, 23], ...",138
6,Pulse,2006,"[[Fantasy, 9], [Drama, 28], [Action, 23], [Thr...",134
7,Blackhat,2015,"[[Drama, 28], [Action, 23], [Crime, 23], [Thri...",130
8,Kite,2014,"[[Drama, 28], [Action, 23], [Crime, 23], [Thri...",130
9,Sherlock: The Abominable Bride,None,"[[Drama, 28], [Action, 23], [Crime, 23], [Thri...",130


<h4>3. Weighted Content Algorithm</h4>

In addition to considering the type of computing similarity, there are more features, such as actors and directors. Let's use the weighted sum to rate suggestions based on the number of actors, genres, and directors they share to improve the score.

1. First select the movie

2. Find movies of the same genre and calculate the sum of the genres

3. Similarly, calculate the actor and director

4. Calculate the total score based on genre proportion 5, actor proportion 3, and director proportion 4

This results in the ranking of other movies with similar characteristics to movie m.

In [17]:
q3 = """MATCH (m:Movie) WHERE m.title = "{}"
MATCH (m)-[:IN_GENRE]->(g:Genre)<-[:IN_GENRE]-(rec:Movie)
WITH m, rec, COUNT(*) AS gs
OPTIONAL MATCH (m)<-[:ACTED_IN]-(a:Actor)-[:ACTED_IN]->(rec)
WITH m, rec, gs, COUNT(a) AS as
OPTIONAL MATCH (m)<-[:DIRECTED]-(d:Director)-[:DIRECTED]->(rec)
WITH m, rec, gs, as, COUNT(d) AS ds
RETURN rec.title AS recommendation, (5*gs)+(3*as)+(4*ds)
AS score ORDER BY score DESC LIMIT 10""".format(favouriteMove)

recommentation3 = session.run(q3)

In [18]:
movieTitle3 = []
score = []

for node in recommentation3:
    movieTitle3.append(node[0])
    score.append(node[1])
    
df3 = pd.DataFrame(columns = ["Movie Title", "Score"])

for i in range(0, len(movieTitle3)):
    df3 = df3.append({"Movie Title":movieTitle3[i],
                      "Score":score[i]}, ignore_index=True)
df3

,Movie Title,Score
0,Inception,30
1,Patlabor: The Movie (Kidô keisatsu patorebâ: T...,30
2,"Double, The",25
3,"Girl Who Played with Fire, The (Flickan som le...",25
4,Sherlock: The Abominable Bride,25
5,Kite,25
6,Source Code,25
7,Blackhat,25
8,Rubber,25
9,Knowing,25


<h4>4.Content-Based Similarity Metrics</h4>

The Jaccard index is a number between 0 and 1, indicating the similarity of the two groups.

    1. The Jaccard index of two identical sets is 1.
    2. If the two sets have no common elements, the Jaccard index is 0.
    3. Jaccard is calculated by dividing the size of the intersection of the two sets by the union of the two sets.
    
We can calculate the Jaccard index of the movie genre set to determine how similar the two movies are.

In [19]:
q4 = """MATCH (m:Movie {{title: "{}"}})-
[:IN_GENRE|:ACTED_IN|:DIRECTED]-(t)<-[:IN_GENRE|:ACTED_IN|:DIRECTED]-(other:Movie)
WITH m, other, COUNT(t) AS intersection, COLLECT(t.name) AS i
MATCH (m)-[:IN_GENRE|:ACTED_IN|:DIRECTED]-(mt)
WITH m,other, intersection,i, COLLECT(mt.name) AS s1
MATCH (other)-[:IN_GENRE|:ACTED_IN|:DIRECTED]-(ot)
WITH m,other,intersection,i, s1, COLLECT(ot.name) AS s2
WITH m,other,intersection,s1,s2
WITH m,other,intersection,s1+filter(x IN s2 WHERE NOT x IN s1) AS union, s1, s2
RETURN other.title, ((1.0*intersection)/SIZE(union))
AS jaccard ORDER BY jaccard DESC LIMIT 10""".format(favouriteMove)

recommentation4 = session.run(q4)

In [20]:
movieTitle4 = []
jaccardIndex = []

for node in recommentation4:
    movieTitle4.append(node[0])
    jaccardIndex.append(node[1])
    
df4 = pd.DataFrame(columns = ["Movie Title", "Jaccard Index"])

for i in range(0, len(movieTitle4)):
    df4 = df4.append({"Movie Title":movieTitle4[i],
                      "Jaccard Index":jaccardIndex[i]}, ignore_index=True)
df4

,Movie Title,Jaccard Index
0,Sherlock: The Abominable Bride,0.454545
1,Inception,0.352941
2,Patlabor: The Movie (Kidô keisatsu patorebâ: T...,0.333333
3,Kite,0.312500
4,"Double, The",0.312500
5,Blackhat,0.312500
6,"Girl Who Played with Fire, The (Flickan som le...",0.312500
7,Whiteout,0.312500
8,Man on Fire,0.312500
9,Cellular,0.312500


<h4>Compare and Contrast for Content-Based Filtering Algorithms</h4>

In [22]:
df = pd.DataFrame(columns = ["Recommend1", "Recommend2", "Recommend3", "Recommend4"])

for i in range(0, len(movieTitle1)):
    df = df.append({"Recommend1":movieTitle1[i],"Recommend2":movieTitle2[i],
                    "Recommend3":movieTitle3[i],"Recommend4":movieTitle4[i]},
                   ignore_index=True)
df

,Recommend1,Recommend2,Recommend3,Recommend4
0,Inception,Motorama,Inception,Sherlock: The Abominable Bride
1,Patlabor: The Movie (Kidô keisatsu patorebâ: T...,Patlabor: The Movie (Kidô keisatsu patorebâ: T...,Patlabor: The Movie (Kidô keisatsu patorebâ: T...,Inception
2,"Double, The","Seven-Per-Cent Solution, The","Double, The",Patlabor: The Movie (Kidô keisatsu patorebâ: T...
3,"Girl Who Played with Fire, The (Flickan som le...",Osmosis Jones,"Girl Who Played with Fire, The (Flickan som le...",Kite
4,Sherlock: The Abominable Bride,Interstate 60,Sherlock: The Abominable Bride,"Double, The"
5,Kite,Sherlock Holmes: A Game of Shadows,Kite,Blackhat
6,Source Code,Pulse,Source Code,"Girl Who Played with Fire, The (Flickan som le..."
7,Blackhat,Blackhat,Blackhat,Whiteout
8,Rubber,Kite,Rubber,Man on Fire
9,Knowing,Sherlock: The Abominable Bride,Knowing,Cellular


The reuslt is not bad. Some movies appear in all recommended lists。